In [1]:
import sys
sys.path.insert(0, os.path.abspath('../'))
import os
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import DataFrame, Row
from functools import reduce
import functools
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
conf=SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

22/12/22 22:26:32 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface enp4s0)
22/12/22 22:26:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/22 22:26:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/22 22:26:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
cluster_file_path = '/home/preacher/Bungee/bnt/data_set/saas_team/cluster_for_back_tracking'

In [9]:
def get_data_from_s3(path):
    df = spark.read.format('csv').option('header','true').load(path)
    print(df.count())
    return df

In [ ]:
# open th cluster File
# get all clusters having length > 30
# for each of the above clusters
#     get of brand names
#     join exact_matches with the ml_input to get the name of all the brands (uuid_A, uuid_b, brand_a, brand_b, product_url_a, product_url_b)


In [12]:
brand_clusters = get_data_from_s3(cluster_file_path)
# cluster here si the cxploded cluseter where one cluster can be identified by using the set id and all brands haveing the same cluster id form one cluster
brand_clusters_set = brand_clusters.groupBy('id').agg(collect_set('brand_id').alias('clusters'))
abnorml_cluster = brand_clusters_set.filter(size(col("clusters"))>30)



5641


In [13]:
abnorml_cluster.show()

+---+--------------------+
| id|            clusters|
+---+--------------------+
|  1|[OLA, Vanderpump ...|
+---+--------------------+



In [24]:
abnorml_exploded_cluster = abnorml_cluster.withColumn( 'brand',explode(abnorml_cluster.clusters)).drop('clusters')

In [25]:
abnorml_exploded_cluster.show()

+---+--------------------+
| id|               brand|
+---+--------------------+
|  1|                 OLA|
|  1|     Vanderpump Pets|
|  1|    Horze Equestrian|
|  1|        Wild Harvest|
|  1|BLUE Life Protection|
|  1|      Supreme Source|
|  1|  John Paul Mitchell|
|  1|            Thyrovet|
|  1|    Polka Dog Bakery|
|  1|       HILLMAN GROUP|
|  1|              Kordon|
|  1|                Awoo|
|  1|            Finntack|
|  1|    Oven Fresh Bites|
|  1|MuttNation On The...|
|  1|            Equizone|
|  1|Vet's Best||Ongua...|
|  1|14 oz powder or 5...|
|  1|       Skout's Honor|
|  1|        Caitec Corp.|
+---+--------------------+
only showing top 20 rows



In [14]:
product_mactches_path = '/home/preacher/Bungee/bnt/data_set/saas_team/exact_matches.csv'
product_mactches = get_data_from_s3(product_mactches_path)

2212437


In [15]:
def create_sku_uuid(uuid):
    arr = uuid.split("<>")
    arr = arr[1:len(arr)]
    return "<>".join(arr)
    
def create_sku_uuid_column(df):
    sku_uuid = udf(create_sku_uuid, StringType())
    res_df = df.withColumn("sku_uuid", sku_uuid(df.uuid))
    return res_df

In [16]:
crawles_data_path = '/home/preacher/Bungee/bnt/data_set/saas_team/crawled_data.csv'
crawled_data = get_data_from_s3(crawles_data_path)
crawled_data = create_sku_uuid_column(crawled_data)

17984206


In [27]:
crawled_data.printSchema()
product_mactches.printSchema()
abnorml_exploded_cluster.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- sku_uuid: string (nullable = true)

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- score: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- match_timestamp: string (nullable = true)
 |-- match_source: string (nullable = true)
 |-- type: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- brand: string (nullable = false)



<h2>Fetching brands for the exact matches</h2>

In [30]:
# getting product info for sku_uuid_a
crawled_data_sku_uuid_a = crawled_data.select('sku_uuid', col('brand').alias('brand_a'), col('product_url').alias('product_url_a'), col('image_url').alias('image_url_a'))
sku_uuid_a_product_info = product_mactches.join(crawled_data_sku_uuid_a, product_mactches.sku_uuid_a == crawled_data_sku_uuid_a.sku_uuid, 'inner').drop('sku_uuid')

# getting product info for sku_uuid_b
crawled_data_sku_uuid_b = crawled_data.select('sku_uuid',col('brand').alias('brand_b'), col('product_url').alias('product_url_b'), col('image_url').alias('image_url_b'))
product_info_matches = sku_uuid_a_product_info.join(crawled_data_sku_uuid_b, sku_uuid_a_product_info.sku_uuid_b == crawled_data_sku_uuid_b.sku_uuid, 'inner').drop('sku_uuid')

In [32]:

product_info_matches.printSchema()
product_info_matches.show(truncate=False, n = 2)

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- score: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- match_timestamp: string (nullable = true)
 |-- match_source: string (nullable = true)
 |-- type: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- brand_a: string (nullable = true)
 |-- product_url_a: string (nullable = true)
 |-- image_url_a: string (nullable = true)
 |-- brand_b: string (nullable = true)
 |-- product_url_b: string (nullable = true)
 |-- image_url_b: string (nullable = true)



+----------------------------------------------------------+----------------------------------------------------------+------------------------------------------+-------------------------------------------+----------+------------------------------+----------------------------+---------------+------------+-----------+-------+----+-----+---+-------+---------------------------------------------------------------------------------+--------------------------------------------------------------------+-------+--------------------------------------------------------------------------------+---------------------------------------------------------+
|uuid_a                                                    |uuid_b                                                    |sku_uuid_a                                |sku_uuid_b                                 |score     |base_source_store             |comp_source_store           |match_timestamp|match_source|type       |segment|year|month|day|brand_a|pro

In [34]:
clusters_product_matches = product_info_matches.join(abnorml_exploded_cluster, product_info_matches.brand_a == abnorml_exploded_cluster.brand, 'inner')

In [35]:
clusters_product_matches.show()

+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+---------------+------------+-----------+-------+----+-----+---+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---+--------------------+
|              uuid_a|              uuid_b|          sku_uuid_a|          sku_uuid_b|     score|   base_source_store|   comp_source_store|match_timestamp|match_source|       type|segment|year|month|day|             brand_a|       product_url_a|         image_url_a|       brand_b|       product_url_b|         image_url_b| id|               brand|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+---------------+------------+-----------+-------+----+-----+---+--------------------+--------------------+--------------------+--------------+--------------------+---

In [37]:
clusters_product_matches.write.mode("overwrite").option("header",True).parquet('/home/preacher/Bungee/bnt/data_set/saas_team/abnormal_cluster_product_matches')

In [36]:
clusters_product_matches.count()

1301856

In [ ]:
clusters_product_matches.write.partitionBy('').saveAsTable('cluster_product_matches')